In [ ]:
import hanlp
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
similarity_threshold = 0.6

def is_topic_change(new_msg, context):
    if not context:
        return False
    last_msg = context[-1]
    emb1 = model.encode(new_msg, convert_to_tensor=True)
    emb2 = model.encode(last_msg, convert_to_tensor=True)
    sim_score = util.pytorch_cos_sim(emb1, emb2).item()
    return sim_score < similarity_threshold

hanlp.load('LARGE_ALBERT_BASE')
HanLP = hanlp.pipeline() \
    .append(hanlp.utils.rules.split_sentence, output_key='sentences') \
    .append(hanlp.load('CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH'), output_key='all') \
    
# message = """
# 戰爭之王
# 戰事​肆虐​至今，​各族​的​英雄​們​一心​精進​戰術。​​戰士​集力量、​統御力​和​對​武器​護甲​的​瞭解於​一身，​於​光榮​之​役​展開​廝殺。​前線​戰士​以​盾牌​防守​封鎖​敵​人，​後方隊​友​則​以​法術​和​弓箭​支援。​其他​戰士們​則​不​使用​盾牌，​以​各式​各樣​致命​武器近​身​攻擊，​怒威​震敵。

# 戰士介紹
# 戰士們​重裝​上陣，​與​敵人​正面​交鋒，​無視於​敵人​在​他們​鎧甲​上​擦身​而​過​的​攻擊。​​他們​運用​變化​多端​的​戰術​與​多款​各式​武器來​保護​較​脆弱​的​隊友。​怒氣​是​戰士們​強力​攻擊​的​原動力，​因此​他們​必須​小心​運用​怒氣，​在​戰鬥​中​發揮​最​大​效益。

# 類型
# 坦克，傷害
# 標準數值
# 生命力, 怒氣
# 可用護甲
# 布甲，皮甲，鎖甲，鎧甲，盾牌
# 可用武器
# 匕首，拳套，斧，錘，劍，長柄武器，法杖
# """
message = """
你好！今天過得怎麼樣？
嗨！還不錯，剛好和朋友去喝了咖啡。你呢？
我也很好，今天早上去跑步了，感覺挺清爽的。
哇，聽起來不錯！你每天都跑步嗎？
是啊，差不多，每週跑三四次。你有做運動嗎？
我喜歡游泳，但最近有點忙，沒時間去游泳池了。
理解！有空我們一起去游泳吧。
好啊，等我有空時就一起去！
"""

data = HanLP(message)["all"]

# print(data)

# for tok_line, pos_line in zip(data["tok/fine"], data["pos/pku"]):
#     nn_line = [token for token, pos in zip(tok_line, pos_line) if pos in {"n", "ns", "nt", "nr"}]
#     print(nn_line)

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Initialize embedding model (multilingual)
embed_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Threshold for topic change detection
SIMILARITY_THRESHOLD = 0.20

# Context buffer
topic_buffer = []

# Check for topic change
def is_topic_change(new_msg, context):  
    if not context:
        return False

    emb_new = embed_model.encode(new_msg, convert_to_tensor=True)
    emb_ctx = embed_model.encode(context, convert_to_tensor=True)
    
    similarity = util.cos_sim(emb_new, emb_ctx.mean(dim=0)).item()
    print(f" ({context[-1]} {new_msg}) -> {similarity}")

    return similarity < SIMILARITY_THRESHOLD

def process_message(message):
    global topic_buffer
    if is_topic_change(message, topic_buffer):
        
        topic_buffer = []
    topic_buffer.append(message)

# Simulate message input
messages = [
    "我上星期去了台北玩。",
    "天氣很熱，但是很好玩。",
    "我還去了故宮，裡面有很多古董。",
    
    "下週學校要開始期中考了。",
    "我數學還沒讀完，有點緊張。",
    
    "最近睡不好，常常熬夜。",
    "醫生建議我早點睡，多運動。",
    
    "我昨天遇到小美。",
    "她說她也去了台北出差。"
]

for msg in messages:
    process_message(msg)
